In [1]:
from sklearn import svm
import os
os.chdir('/Users/ritwikdeshpande/DLH/Project/CS598_DLH_Project/')
from dataset.preprocessing.word2vec_embeddings_gen import Word2VecFeatureGeneration
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import pandas as pd
import csv
import numpy as np
import collections


In [2]:
class SVM:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.svm = svm.SVC(kernel='linear')
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def train(self):
        
        self.svm.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.svm.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        # print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [3]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

In [5]:
column_headings = ["Morbidity Class", "SVM_Macro F1", "SVM_Micro F1"]

with open("./results/word-embeddings-features/performance_SVM.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = Word2VecFeatureGeneration(train_preprocessed_df, morbidity).word2vec_matrix_gen()
    X = np.average(X, axis=1)
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            svm_obj = SVM(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
            svm_obj.train()

            f1_macro, f1_micro = svm_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]

    with open("./results/word-embeddings-features/performance_SVM.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_SVM.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)
    

(572, 3570, 100) (572,) Counter({0.0: 502, 1.0: 70})
Macro F1 score: 0.46296296296296297 and Micro F1 Score 0.8620689655172413
Macro F1 score: 0.4727272727272727 and Micro F1 Score 0.896551724137931
Macro F1 score: 0.46226415094339623 and Micro F1 Score 0.8596491228070176
Macro F1 score: 0.45714285714285713 and Micro F1 Score 0.8421052631578947
Macro F1 score: 0.4770642201834862 and Micro F1 Score 0.9122807017543859
Macro F1 score: 0.4770642201834862 and Micro F1 Score 0.9122807017543859
Macro F1 score: 0.4770642201834862 and Micro F1 Score 0.9122807017543859
Macro F1 score: 0.46226415094339623 and Micro F1 Score 0.8596491228070176
Macro F1 score: 0.46728971962616817 and Micro F1 Score 0.8771929824561403
Macro F1 score: 0.45714285714285713 and Micro F1 Score 0.8421052631578947
Macro F1 score: 0.4672986632039369 and Micro F1 Score 0.8776164549304294
(548, 3570, 100) (548,) Counter({1.0: 325, 0.0: 223})
Macro F1 score: 0.3452380952380952 and Micro F1 Score 0.5272727272727272
Macro F1 sco